# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
M. Samland  ->  M. Samland  |  ['M. Samland']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']


Arxiv has 51 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.04775
extracting tarball to tmp_2508.04775... done.
Retrieving document from  https://arxiv.org/e-print/2508.05052
extracting tarball to tmp_2508.05052...

 done.
Retrieving document from  https://arxiv.org/e-print/2508.05155
extracting tarball to tmp_2508.05155... done.


T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
M. Samland  ->  M. Samland  |  ['M. Samland']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']


Issues with the citations
list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.05155-b31b1b.svg)](https://arxiv.org/abs/2508.05155) | **MINDS. Cha Hα 1, a brown dwarf with a hydrocarbon-rich disk**  |
|| M. Morales-Calderón, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>M. Samland</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-08-08*|
|*Comments*| *16 pages, 14 figures*|
|**Abstract**|            Context. Recent JWST observations have shown that brown dwarfs (BD) are chemically rich, offering valuable insights into giant planet formation. Aims. As part of the MIRI mid-INfrared Disk Survey (MINDS) JWST guaranteed time program, we aim to characterize the gas and dust composition of the disk around the brown dwarf [NC98] Cha HA 1, hereafter Cha H$\alpha$ 1, in the mid-infrared. Methods. We obtain data from the MIRI Medium Resolution Spectrometer (MRS) from 4.9 to 28$\mu$m. We use the dust fitting tool DuCK to investigate the dust composition and grain sizes while we identify and fit molecular emission using slab models. Results. Compared with disks around very low mass stars, clear silicate emission features are seen in this BD disk. In addition, JWST reveals a plethora of hydrocarbons, including C$_2$H$_2$, $^{13}$CCH$_2$, CH$_3$, CH$_4$, C$_2$H$_4$, C$_4$H$_2$, C$_3$H$_4$, C$_2$H$_6$, and C$_6$H$_6$ which suggest a disk with a gas C/O > 1. Additionally, we detect CO$_2$, $^{13}$CO$_2$, HCN, H$_2$, and H$_2$O. CO and OH are absent from the spectrum. The dust is dominated by large $\sim$4 $\mu$m size amorphous silicates (MgSiO$_3$). We infer a small dust mass fraction ($>$10$\%$) of 5 $\mu$m size crystalline forsterite. We do not detect polycyclic aromatic hydrocarbons. Conclusions. Cha H$\alpha$ 1 shows the most diverse chemistry seen to date in a BD protoplanetary disk, consisting of a strong dust feature, 12 carbon-bearing molecules plus H$_2$, and water. The diverse molecular environment offers a unique opportunity to test our understanding of BD disks chemistry and how it affects the possible planets forming in them.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.04775-b31b1b.svg)](https://arxiv.org/abs/2508.04775) | **Resolution-Corrected White Dwarf Gravitational Redshifts Validate SDSS-V Wavelength Calibration and Enable Accurate Mass-Radius Tests**  |
|| S. M. Arseneau, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-08-08*|
|*Comments*| *17 pages, 11 figures, 2 tables, accepted for publication in ApJ*|
|**Abstract**|            Leveraging the large sample size of low-resolution spectroscopic surveys to constrain white dwarf stellar structure requires an accurate understanding of the shapes of hydrogen absorption lines, which are pressure broadened by the Stark effect. Using data from both the Sloan Digital Sky Survey and the Type Ia Supernova Progenitor Survey, we show that substantial biases (5-15 km/s) exist in radial velocity measurements made from observations at low spectral resolution relative to similar measurements from high-resolution spectra. Our results indicate that the physics of line formation in high-density plasmas, especially in the wings of the lines, are not fully accounted for in state-of-the-art white dwarf model atmospheres. We provide corrections to account for these resolution-induced redshifts in a way that is independent of an assumed mass-radius relation, and we demonstrate that statistical measurements of gravitational redshift with these corrections yield improved agreement with theoretical mass-radius relations. Our results provide a set of best practices for white dwarf radial velocity measurements from low-resolution spectroscopy, including those from the Sloan Digital Sky Survey, the Dark Energy Spectroscopic Instrument, the 4-meter Multi-Object Spectroscopic Telescope, and the Wide-Field Multiplexed Spectroscopic Facility.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.05052-b31b1b.svg)](https://arxiv.org/abs/2508.05052) | **The Size Evolution and the Size-Mass Relation of Lyman-Alpha Emitters across $3 \lesssim z < 7$ as Observed by JWST**  |
|| Q. Song, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-08-08*|
|*Comments*| *16 pages, 10 figures, 2 tables. Comments are welcome*|
|**Abstract**|            Understanding the morphological structures of Lyman-alpha emitters (LAEs) is crucial for unveiling their formation pathways and the physical origins of Ly$\alpha$ emission. However, the evolution of their sizes and structural scaling relations remains debated. In this study, we analyze a large sample of 876 spectroscopically confirmed LAEs at $3 \lesssim z < 7$, selected from the MUSE, VANDELS, and CANDELSz7 surveys in the GOODS-S, UDS, and COSMOS fields. Utilizing deep, high-resolution near-infrared images from the James Webb Space Telescope (JWST), we measure their rest-frame UV and optical V-band effective radii ($R_{\rm e}$) through two-dimensional Sérsic profile fitting. Our results show that these LAEs are generally compact with weak size evolution, following $R_{\rm e,UV} \propto (1 + z)^{-0.91 \pm 0.10}$ and $R_{\rm e,V} \propto (1 + z)^{-0.93 \pm 0.18}$, respectively. Their UV and optical sizes are statistically comparable, indicating negligible UV-to-optical color gradients. For the first time, we establish the rest-frame optical size-mass relation for LAEs at $z>3$, finding slopes comparable to typical star-forming galaxies (SFGs), but with slightly smaller sizes at a given stellar mass. These results provide important clues for understanding the formation mechanisms and structural evolution of LAEs in the early universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.05155/./Figures/models_large_zoom.png', 'tmp_2508.05155/./Figures/Xis.png', 'tmp_2508.05155/./Figures/BandsOffsets_v2.png']
copying  tmp_2508.05155/./Figures/models_large_zoom.png to _build/html/
copying  tmp_2508.05155/./Figures/Xis.png to _build/html/
copying  tmp_2508.05155/./Figures/BandsOffsets_v2.png to _build/html/
exported in  _build/html/2508.05155.md
    + _build/html/tmp_2508.05155/./Figures/models_large_zoom.png
    + _build/html/tmp_2508.05155/./Figures/Xis.png
    + _build/html/tmp_2508.05155/./Figures/BandsOffsets_v2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}{\textcolor{purple}}$
$\newcommand{\mum}{\mum\xspace}$
$\newcommand{\referee}{\textcolor{red}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Macc}{\dot{M}_\mathrm{acc}}$
$\newcommand{\chaha1}{Cha~H\alpha~1}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# MINDS. Cha H$\alpha$ 1, a brown dwarf with a hydrocarbon-rich disk.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.05155-b31b1b.svg)](https://arxiv.org/abs/2508.05155)<mark>Appeared on: 2025-08-08</mark> -  _16 pages, 14 figures_

</div>
<div id="authors">

M. Morales-Calderón, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>M. Samland</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The chemistry of disks around brown dwarfs (BDs) remain largely unexplored due to their faintness. Despite the efforts performed with Spitzer, we have far less understanding of planet formation, chemical composition, disk structure, and evolution in disks around BDs compared to their more massive counterparts (T Tauri and Herbig Ae/Be stars), which are more readily studied due to their greater brightness. Recent JWST observations, with up to an order of magnitude improvement in both spectral and spatial resolution, have shown that these systems are chemically rich, offering valuable insights into giant planet formation. As part of the MIRI mid-INfrared Disk Survey (MINDS) JWST guaranteed time program, we aim to characterize the gas and dust composition of the disk around the brown dwarf [ NC98 ] Cha HA 1, hereafter $\chaha$ 1, in the mid-infrared. We obtain data from the MIRI Medium Resolution Spectrometer (MRS) from 4.9 to 28 $\mum$ ( $R\sim$ 1500 - 3500; FWHM $\sim$ 0.2" - 1.2"). We use the dust fitting tool DuCK to investigate the dust composition and grain sizes while we identify and fit molecular emission in the spectrum using slab models. Compared with disks around very low mass stars, clear silicate emission features are seen in this BD disk. In addition, JWST reveals a plethora of hydrocarbons, including $C_2$ $H_2$ , $^{13}$ $CCH_2$ , $CH_3$ , $CH_4$ , $C_2$ $H_4$ , $C_4$ $H_2$ ,  $C_3$ $H_4$ , $C_2$ $H_6$ ,  and $C_6$ $H_6$ which suggest a disk with a gas C/O > 1. Additionally, we detect $CO_2$ , $^{13}$ $CO_2$ , HCN, $H_2$ , and $H_2$ O. CO and OH are absent from the spectrum.The dust is dominated by large $\sim$ 4 $\mum$ size amorphous silicates ($MgSiO_3$ ). We infer a small dust mass fraction ( $>$ 10 $\%$ ) of 5 $\mum$ size crystalline forsterite. We do not detect polycyclic aromatic hydrocarbons. The mid-infrared spectrum of $\chaha$ 1 shows the most diverse chemistry seen to date in a BD protoplanetary disk, consisting of a strong dust feature, 12 carbon-bearing molecules plus $H_2$ , and water. The diverse molecular environment offers a unique opportunity to test our understanding of BD disks chemistry and how it affects the possible planets forming in them.

</div>

<div id="div_fig1">

<img src="tmp_2508.05155/./Figures/models_large_zoom.png" alt="Fig12" width="100%"/>

**Figure 12. -** $\chaha$1 spectrum, with the JWST-MIRI continuum subtracted data (black) compared to the stacked emission from the slab models for $C_2$$H_2$ + $^{13}$$CCH_2$, HCN, $CO_2$ + $^{13}$$CO_2$, $C_6$$H_6$, $C_3$$H_4$, $C_4$$H_2$, $C_2$$H_4$, $C_2$$H_6$, $CH_4$, $H_2$O, and $CH_3$. The parameters of the slab models shown can be found in Table \ref{Tab:molecules} and  \ref{Tab:App:molecules}. The $H_2$ lines are also marked. Horizontal lines represent the windows in which the $\chi^2$ for each fitting has been evaluated. Note that the y axis are different for each panel. (*Fig:models*)

</div>
<div id="div_fig2">

<img src="tmp_2508.05155/./Figures/Xis.png" alt="Fig11" width="100%"/>

**Figure 11. -** $\chi^2$ maps for HCN, $CO_2$ +  $^{13}$$CO_2$, $C_6$$H_6$, $C_2$$H_4$, $C_2$$H_6$, $CH_4$, and  water. The color scale shows the $\chi^2_{min}$/$\chi^2$. The best fit model ($\chi^2_{min}$/$\chi^2$ = 1) is marked with a red circle. A red contour denotes the emitting region radius as listed in Table \ref{Tab:molecules}. Gray contours show the emitting radii in au while white contours represent the 1$\sigma$, 2$\sigma$, and 3$\sigma$ levels.
 (*Fig:xis*)

</div>
<div id="div_fig3">

<img src="tmp_2508.05155/./Figures/BandsOffsets_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** MIRI $\chaha$1 reduced spectra. Blue, yellow, and red colors denote the SHORT, MEDIUM and LONG bands respectively for each channel of the ddither approach.  The gray line shows the sdither reduction and the blue spectrum is the final flux-corrected one in Ch. 4. The green line corresponds to the Spitzer $\chaha$1 low resolution spectrum obtained from CASSIS \citep{Lebouteiller11}. (*Fig:spec*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.05155"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

135  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
